In [1]:
params = {'slope': 7, 'limit': 1623, 'samples': 423, 'sigma': 1.0056550229113084, 'subwidth': 21, 'subweight': 29, 'mask_size': 5, 'iterations': 7, 'min_size': 143}

In [10]:
import pathlib

import optuna
from skimage import filters
from scipy.ndimage import gaussian_filter
from sklearn.metrics import f1_score
import colorcorrect.algorithm as cca
import mahotas as mh
import cellseg.src.image_worker as iw
import cellseg.src.convert_worker as cw
import cellseg.src.shape_descriptors as sd
import numpy as np
import pandas as pd

results_df = pd.DataFrame(columns=["name", "Dice_coefficient"])

for bal_image in pathlib.Path("Images/BAL_images/PoC/").glob("*.png"):
    print(bal_image)
    img = mh.imread(bal_image)
    img_c_corrected = cca.automatic_color_equalization(img, slope=params['slope'], limit=params['limit'], samples=params['samples'])
    
    img_blurred = gaussian_filter(img_c_corrected, sigma=params['sigma'])
    
    RGB_balanced = cca.luminance_weighted_gray_world(img_blurred, subwidth=params['subwidth'], subheight=params['subweight'])
    r2, g2, b2 = RGB_balanced[:, :, 0], RGB_balanced[:, :, 1], RGB_balanced[:, :, 2]
    
    thresholds = filters.threshold_multiotsu(g2)
    regions = np.digitize(g2, bins=thresholds)
    bin_cyto_nuclei = cw.convert_labeled_to_bin(regions, background=2)
    bin_cyto_nuclei = iw.close_holes_remove_noise(bin_cyto_nuclei, mask_size=params['mask_size'], iterations=params['iterations'])
    
    img_labeled_cytoplasm, nr_cytoplasm = mh.label(bin_cyto_nuclei)
    img_labeled_cytoplasm = iw.remove_small_regions(img_labeled_cytoplasm, min_size=params['min_size'])
    img_cytoplasm_nuclei = mh.labeled.remove_bordering(img_labeled_cytoplasm)
    
    label_file = f"Images/labels/cytoplasm/{bal_image.stem}.txt"
    
    cytoplasm_labels = np.loadtxt(label_file, dtype=int)
    
    f1 = f1_score(cytoplasm_labels.flatten(), img_cytoplasm_nuclei.flatten(), average="micro")
    
    # results_df = results_df.append({"name": bal_image.stem, "Dice_coefficient": f1}, ignore_index=True)
    result = pd.DataFrame({"name": bal_image.stem, "Dice_coefficient": f1}, index=[0])
    print(result)
    results_df = pd.concat([results_df, result], ignore_index=True)

results_df.to_csv("results.csv", index=False)

Images/BAL_images/PoC/2023_12_14_image_005.png
                   name  Dice_coefficient
0  2023_12_14_image_005          0.801539
Images/BAL_images/PoC/2023_12_14_image_000.png


/tmp/ipykernel_398099/3977999630.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, result], ignore_index=True)


                   name  Dice_coefficient
0  2023_12_14_image_000          0.893167
Images/BAL_images/PoC/2023_12_14_image_017.png
                   name  Dice_coefficient
0  2023_12_14_image_017          0.874456
Images/BAL_images/PoC/2023_12_14_image_018.png
                   name  Dice_coefficient
0  2023_12_14_image_018          0.879137
Images/BAL_images/PoC/2023_12_14_image_032.png
                   name  Dice_coefficient
0  2023_12_14_image_032          0.791995
Images/BAL_images/PoC/2023_12_14_image_025.png
                   name  Dice_coefficient
0  2023_12_14_image_025          0.930487
Images/BAL_images/PoC/2023_12_14_image_038.png
                   name  Dice_coefficient
0  2023_12_14_image_038          0.880147
Images/BAL_images/PoC/2023_12_14_image_031.png
                   name  Dice_coefficient
0  2023_12_14_image_031          0.852582
Images/BAL_images/PoC/2023_12_14_image_008.png
                   name  Dice_coefficient
0  2023_12_14_image_008          0.896922